In [1]:
import pandas as pd
import re
import itertools

In [2]:
import ast

In [3]:
df_final = pd.DataFrame()

In [4]:
import os

# folder path
# dir_path = r'/home/trishalaswain/Downloads/ph-febData/'
dir_path = r'/home/trishalaswain/PH_01_to_31_jan/'


# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    # if os.path.isfile(os.path.join(dir_path, path)) and str(path).endswith(".tsv") and not str(path).endswith("HCPCS.tsv") and not str(path).endswith("CPT.tsv") and not str(path).endswith("CM.tsv")  and not str(path).endswith("EANDM.tsv"):
    if os.path.isfile(os.path.join(dir_path, path)) and str(path).endswith("31.xlsx"):
        res.append(dir_path + path)

In [5]:
res

['/home/trishalaswain/PH_01_to_31_jan/PH-LAMC-Los Alamos Medical Center-2023-01-01-2023-01-31.xlsx',
 '/home/trishalaswain/PH_01_to_31_jan/PH-EHMC-Encino Hospital Medical Center-2023-01-01-2023-01-31.xlsx',
 '/home/trishalaswain/PH_01_to_31_jan/PH-SDCH-San Dimas Community Hospital-2023-01-01-2023-01-31.xlsx',
 '/home/trishalaswain/PH_01_to_31_jan/PH-SOH-Sherman Oaks Hospital-2023-01-01-2023-01-31.xlsx',
 '/home/trishalaswain/PH_01_to_31_jan/PH-AHE-Alvarado Hospital EMA-2023-01-01-2023-01-31.xlsx',
 '/home/trishalaswain/PH_01_to_31_jan/PH-CELA-East Los Angeles Doctors Hospital-2023-01-01-2023-01-31.xlsx',
 '/home/trishalaswain/PH_01_to_31_jan/PH-LPIH-La Palma Intercommunity Hospital-2023-01-01-2023-01-31.xlsx',
 '/home/trishalaswain/PH_01_to_31_jan/PH-RMHG-Redondo Memorial Hospital of Gardena-2023-01-01-2023-01-31.xlsx',
 '/home/trishalaswain/PH_01_to_31_jan/PH-HEMAG-Hollywood EMA Group-2023-01-01-2023-01-31.xlsx',
 '/home/trishalaswain/PH_01_to_31_jan/PH-SRMC-Shasta Regional Medical Ce

In [6]:
def sugg_rej(df1):
    for index, row in df1.iterrows():
        for code in row['EANDM_Suggested_code']:
            if str(code) in str(row['EANDM_Final_code']):
    #           incrementing accepted count if code found in both suggested and final
                df1.at[index,'EANDM_Updated_Accepted_Count'] += 1
            if str(code) not in str(row['EANDM_Final_code']):
    #           incrementing rejected count if code found suggested but not in final
                df1.at[index,'EANDM_Updated_Rejected_Count'] += 1

In [7]:
def add(df1):
    for index, row in df1.iterrows():
        for fcode in row['EANDM_Final_code']:
            if str(fcode) not in str(row['EANDM_Suggested_code']):
    #           incrementing added count if code found final but not in suggested
                df1.at[index,'EANDM_Updated_Added_Count'] += 1

In [14]:
def emsugg_filtered(df1):
    df1['EANDM_Suggested_code'] = None
    for ind, row in df1.iterrows():
        # print(type(row['EANDM_Accepted']))
        if len(row["EANDM_Accepted"]) == 2:
            accepted = ""
        else:
            accepted = row["EANDM_Accepted"][1:-1]

        if len(row["EANDM_Rejected"]) == 2:
            rejected = ""
        else:
            rejected = row["EANDM_Rejected"][1:-1]
            
        if len(accepted) == 0:
            li = []
        else:
            li = ast.literal_eval(accepted)
            li = [li]

        if len(rejected) == 0:
            li2 = []
        else:
            li2 = ast.literal_eval(rejected)
            li2 = [li2]

        li3 = []
        for item in li:
            li3.append(item)
        for item2 in li2:
            li3.append(item2)
            
        li3 = []
        for item in li:
            if type(item) is tuple:
                for x in item:
                    li3.append(x)
                    
            else:
                li3.append(item)

        for item2 in li2:
            if type(item2) is tuple:
                for x in item2:
                    li3.append(x)
            else: 
                li3.append(item2)
                
        if 99282 in li3:
            li3.remove(99282)
        if 99281 in li3:
            li3.remove(99281)
            
        # if 99282 in li3 and len(li3) != 1:
        #     print(row)
        #     li3.remove(99282)

        df1.at[ind,'EANDM_Suggested_code'] = li3

In [15]:
def emfinal_filtered(df1):
    df1['EANDM_Final_code'] = None
    for ind, row in df1.iterrows():
        if len(row["EANDM_Accepted"]) == 2:
            accepted = ""
        else:
            accepted = row["EANDM_Accepted"][1:-1]
            
        if len(row["EANDM_Added"]) == 2:
            added = ""
        else:
            added = row["EANDM_Added"][1:-1]

        if len(accepted) == 0:
            li = []
        else:
            li = ast.literal_eval(accepted)
            li = [li]

        if len(added) == 0:
            li2 = []
        else:
            li2 = ast.literal_eval(added)
            li2 = [li2]

        li3 = []
        for item in li:
            if type(item) is tuple:
                for x in item:
                    li3.append(x)
            else:
                li3.append(item)

        for item2 in li2:
            if type(item2) is tuple:
                for x in item2:
                    li3.append(x)
            else: 
                li3.append(item2)
        # li3.remove(
       
        df1.at[ind,'EANDM_Final_code'] = li3

In [17]:
i = 0
for file in res:

    # filename = re.findall("SCP-\d.*", file)
    filename = re.findall("PH-.*", file)
    # df = pd.read_excel('/home/trishalaswain/PH_01_to_31_jan/PH-CHMC-Centinela Hospital Medical Center-2023-01-01-2023-01-31.xlsx')

    print(str(filename))
    df = pd.read_excel(file, index_col = False)
    # df
    # Eliminating rows with 0 Accepted, 0 Rejected & 0 Added counts
    df = df[(df['EANDM_Accepted_Count'] != 0) | (df['EANDM_Rejected_Count'] != 0) | (df['EANDM_Added_Count'] != 0)]
    df = df.reset_index()
    df1 = df[["facilityCode","facilityName","accountNumber","EANDM_Final","EANDM_Suggested",
              "EANDM_Accepted","EANDM_Rejected","EANDM_Added",
              "EANDM_Accepted_Count","EANDM_Rejected_Count","EANDM_Added_Count"]].reset_index()

    emsugg_filtered(df1)
    emfinal_filtered(df1)

    df1['EANDM_Updated_Accepted_Count'] = 0
    df1['EANDM_Updated_Rejected_Count'] = 0
    df1['EANDM_Updated_Added_Count'] = 0
    # print(df1)
    sugg_rej(df1)
    add(df1)

    old_sum = df1['EANDM_Accepted_Count'].sum() + df1['EANDM_Added_Count'].sum()
    print("Old Accepted + Added\t" +  str(old_sum))

    new_sum = df1['EANDM_Updated_Accepted_Count'].sum() + df1['EANDM_Updated_Added_Count'].sum()
    print("New Accepted + Added\t" +  str(new_sum))

    # old accuracy measures
    precision = df1['EANDM_Accepted_Count'].sum() / (df1['EANDM_Accepted_Count'].sum() + df1['EANDM_Rejected_Count'].sum())
    recall = df1['EANDM_Accepted_Count'].sum() / (df1['EANDM_Accepted_Count'].sum() + df1['EANDM_Added_Count'].sum())
    f1_score = (2 * precision * recall)/(precision + recall)
    # new accuracy measures
    new_precision = df1['EANDM_Updated_Accepted_Count'].sum() / (df1['EANDM_Updated_Accepted_Count'].sum() + df1['EANDM_Updated_Rejected_Count'].sum())
    new_recall = df1['EANDM_Updated_Accepted_Count'].sum() / (df1['EANDM_Updated_Accepted_Count'].sum() + df1['EANDM_Updated_Added_Count'].sum())
    new_f1_score = (2 * new_precision * new_recall)/(new_precision + new_recall)


    df_final.at[i,'Facility_name'] = str(filename).strip(".tsv")
    df_final.at[i,'Precision'] = round(precision,4)
    df_final.at[i,'New Precision'] = round(new_precision,4)
    df_final.at[i,'Recall'] = round(recall,4)
    df_final.at[i,'New Recall'] = round(new_recall,4)
    df_final.at[i,'F1-Score'] = round(f1_score,4)
    df_final.at[i,'New F1_SCore'] = round(new_f1_score,4)

    df_final.at[i,'initial_accepted_counts'] = df1.EANDM_Accepted_Count.sum()
    df_final.at[i,'initial_added_counts'] = df1.EANDM_Added_Count.sum()
    df_final.at[i,'initial_rejected_counts'] = df1.EANDM_Rejected_Count.sum()

    df_final.at[i,'updated_accepted_counts'] = df1.EANDM_Updated_Accepted_Count.sum()
    df_final.at[i,'updated_added_counts'] = df1.EANDM_Updated_Added_Count.sum()
    df_final.at[i,'updated_rejected_counts'] = df1.EANDM_Updated_Rejected_Count.sum()
    i += 1

    # print(filename)
    # print("old accuracy measures")
    # print("Precision\t" + str(precision))
    # print("Recall\t\t" + str(recall))
    # print("F1-score\t" + str(f1_score))
    # print("new accuracy measures")
    # print("Precision\t" + str(new_precision))
    # print("Recall\t\t" + str(new_recall))
    # print("F1-score\t" + str(new_f1_score))
    print("\n\n")

['PH-LAMC-Los Alamos Medical Center-2023-01-01-2023-01-31.xlsx']
Old Accepted + Added	537
New Accepted + Added	537



['PH-EHMC-Encino Hospital Medical Center-2023-01-01-2023-01-31.xlsx']
Old Accepted + Added	724
New Accepted + Added	724



['PH-SDCH-San Dimas Community Hospital-2023-01-01-2023-01-31.xlsx']
Old Accepted + Added	1027
New Accepted + Added	1027



['PH-SOH-Sherman Oaks Hospital-2023-01-01-2023-01-31.xlsx']
Old Accepted + Added	1692
New Accepted + Added	1692



['PH-AHE-Alvarado Hospital EMA-2023-01-01-2023-01-31.xlsx']
Old Accepted + Added	2721
New Accepted + Added	2721



['PH-CELA-East Los Angeles Doctors Hospital-2023-01-01-2023-01-31.xlsx']
Old Accepted + Added	1061
New Accepted + Added	1061



['PH-LPIH-La Palma Intercommunity Hospital-2023-01-01-2023-01-31.xlsx']
Old Accepted + Added	1285
New Accepted + Added	1285



['PH-RMHG-Redondo Memorial Hospital of Gardena-2023-01-01-2023-01-31.xlsx']
Old Accepted + Added	2869
New Accepted + Added	2869



['PH-HEMAG-Hollywood

In [18]:
# df_final

In [19]:
# df1

In [20]:
# df1.to_excel("/home/trishalaswain/Downloads/excel.xlsx")

In [21]:
# df1.EANDM_Suggested.unique()

In [22]:
# file_name = 'accuracy-analysis.xlsx'
  
# saving the excel
df_final.to_excel('/home/trishalaswain/Downloads/ph-accuracy-without-99281_99282-JanData.xlsx')

In [ ]:
# df_final

In [ ]:
# df.shape

In [ ]:
# Eliminating rows with 0 Accepted, 0 Rejected & 0 Added counts
# df = df[(df["EANDM_Accepted_Count"] != 0) | (df["EANDM_Rejected_Count"] != 0) | (df["EANDM_Added_Count"] != 0)]

In [ ]:
# df.shape

In [ ]:
# df = df.reset_index()

In [ ]:
# df

In [ ]:
# df1 = df[["facilityCode","facilityName","accountNumber","EANDM_Final2","EANDM_Suggested","EANDM_Accepted_Count","EANDM_Rejected_Count","EANDM_Added_Count"]].reset_index()

In [ ]:
# df1

In [ ]:
# df1.EANDM_Suggested.unique()

In [ ]:
# df1['EANDM_Suggested_code'] = None

In [ ]:
# for index, row in df1.iterrows():
#     li = []
#     i = str(row["EANDM_Suggested"])
#     if "99285" in i:
#         li.append(99285)
#     if "99284" in i:
#         li.append(99284)
#     if "99283" in i:
#         li.append(99283)
#     if "99282" in i:
#         li.append(99282)
#     # if "99281" in i:
#         # li.append(99281)
#     if "99291" in i:
#         li.append(99291)
#     if "99292" in i:
#         li.append(99292)
    
#     df1.at[index,'EANDM_Suggested_code'] = li

In [ ]:
# df1['EANDM_Final_code'] = None

In [ ]:
# for index, row in df1.iterrows():
#     li = []
#     i = str(row["EANDM_Final2"])
#     if "99285" in i:
#         li.append(99285)
#     if "99284" in i:
#         li.append(99284)
#     if "99283" in i:
#         li.append(99283)
#     if "99282" in i:
#         li.append(99282)
#     if "99281" in i:
#         li.append(99281)
#     if "99291" in i:
#         li.append(99291)
#     if "99292" in i:
#         li.append(99292)
    
#     df1.at[index,'EANDM_Final_code'] = li

In [ ]:
# df1

In [ ]:
# df1['EANDM_Updated_Accepted_Count'] = 0
# df1['EANDM_Updated_Rejected_Count'] = 0
# df1['EANDM_Updated_Added_Count'] = 0

In [ ]:
# df1['EANDM_Updated_Accepted_Count'].info

In [ ]:
# def sugg_rej(df1):
#     for index, row in df1.iterrows():
#         for code in row['EANDM_Suggested_code']:
#             if str(code) in str(row['EANDM_Final_code']):
#     #           incrementing accepted count if code found in both suggested and final
#                 df1.at[index,'EANDM_Updated_Accepted_Count'] += 1
#             if str(code) not in str(row['EANDM_Final_code']):
#     #           incrementing rejected count if code found suggested but not in final
#                 df1.at[index,'EANDM_Updated_Rejected_Count'] += 1

In [ ]:
# def add(df1):
#     for index, row in df1.iterrows():
#         for fcode in row['EANDM_Final_code']:
#             if str(fcode) not in str(row['EANDM_Suggested_code']):
#     #           incrementing added count if code found final but not in suggested
#                 df1.at[index,'EANDM_Updated_Added_Count'] += 1

In [ ]:
# df1

In [ ]:
# # old accuracy measures
# precision = df1['EANDM_Accepted_Count'].sum() / (df1['EANDM_Accepted_Count'].sum() + df1['EANDM_Rejected_Count'].sum())
# recall = df1['EANDM_Accepted_Count'].sum() / (df1['EANDM_Accepted_Count'].sum() + df1['EANDM_Added_Count'].sum())
# f1_score = (2 * precision * recall)/(precision + recall)

In [ ]:
# # new accuracy measures
# new_precision = df1['EANDM_Updated_Accepted_Count'].sum() / (df1['EANDM_Updated_Accepted_Count'].sum() + df1['EANDM_Updated_Rejected_Count'].sum())
# new_recall = df1['EANDM_Updated_Accepted_Count'].sum() / (df1['EANDM_Updated_Accepted_Count'].sum() + df1['EANDM_Updated_Added_Count'].sum())
# new_f1_score = (2 * new_precision * new_recall)/(new_precision + new_recall)

In [ ]:
# print("old accuracy measures")
# print("Precision\t" + str(precision))
# print("Recall\t\t" + str(recall))
# print("F1-score\t" + str(f1_score))

In [ ]:
# print("new accuracy measures")
# print("Precision\t" + str(new_precision))
# print("Recall\t\t" + str(new_recall))
# print("F1-score\t" + str(new_f1_score))